In [2]:
#Importing 
from importlib.machinery import SourceFileLoader
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt

#Settings for MOBIUS and its PythonWrapper
wr = SourceFileLoader("mobius", r"../mobius.py").load_module()
cu = SourceFileLoader("mobius_calib_uncert_lmfit", r"../mobius_calib_uncert_lmfit.py").load_module()

FileNotFoundError: [Errno 2] No such file or directory: '../mobius.py'

In [1]:
wr.initialize("../../Applications/eDNA/edna.dll")

NameError: name 'wr' is not defined

In [ ]:
ds = wr.DataSet.setup_with_blank_index_sets("input_template.dat")

In [ ]:
species = ["Test"]
ds.set_indexes("Fish species", species)

In [ ]:
dists = np.arange(10000, 38000, 20)
n_reaches = len(dists)

reaches = []
for i in range(n_reaches) :
    prev = []
    if i > 0 :
        prev = [str(i-1)]
    pair = [str(i), prev]
    reaches.append(pair)

ds.set_branch_indexes("Reach", reaches)

In [ ]:
ds.read_parameters("templatepars.dat")   # Reads just (some) parameters, not index sets

In [ ]:
ts = 2*3600
#print("input time steps are %d" % ts)
fls = .5

scale = fls + fls*np.arange(0, n_reaches, 1/float(n_reaches))
vol = 5e4*np.ones(ts)
flow = 2*np.ones(ts)
for i in range(n_reaches) :
    ds.set_input_series("Reach volume", [str(i)], vol*scale[i])
    ds.set_input_series("Reach flow",   [str(i)], flow*scale[i])

In [ ]:
#dists = np.arange(10000, 38000, 20)
for i in range(n_reaches) :
    ds.set_parameter_double("Distance from outlet", [str(i)], dists[99-i])

In [ ]:
for sp in species :
    ds.set_parameter_double("Biomass at outlet", [sp], 100)
    #ds.set_parameter_double("Biomass distribution linear coeff", [sp], 0.02)
    ds.set_parameter_double("Biomass distribution exponential coeff", [sp], -0.002)

In [ ]:
ds.set_parameter_double("eDNA shedding rate per fish biomass", [], 0.1)
ds.set_parameter_double("eDNA decay rate", [], 0.01)

In [ ]:
ds.run_model()

In [ ]:
ts_last = ds.get_last_timesteps()
ts_all = ds.get_next_timesteps()
print(ts_all)
sp = species[0]
res = np.zeros(n_reaches)
res_over_time = np.zeros(ts_all)
res_over_time = ds.get_result_series("eDNA concentration", [sp, "99"])
#res_mat = np.zeros((n_reaches,ts_all))
for i in range(n_reaches) :
    res[99-i] = ds.get_result_series("eDNA concentration", [sp, str(i)])[ts_last-1]
    #res_mat[99-i][:] = ds.get_result_series("Fish biomass", [sp, str(i)])
    #res[99-i] = ds.get_result_series("eDNA mass", [sp, str(i)])[ts-1]
plt.plot(res)
#plt.plot(res_over_time)
#np.savetxt("test.csv", res_mat, delimiter=",")
#plt.imshow(res_mat, cmap='hot')
#plt.show()